<a href="https://colab.research.google.com/github/safeincraziworld/DAblog/blob/master/grp%20proj1.8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving fake_job_postings.csv to fake_job_postings (1).csv


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model, tree, model_selection, ensemble, svm, metrics, naive_bayes
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df=pd.read_csv('fake_job_postings.csv')

In [ ]:
# drop columns with more than 50% missing data
df.drop(columns=['department', 'salary_range', 'industry'], inplace = True)

In [ ]:
#-------------------start cleaning for location column

#--------1. split the columns to country, state, and city

# to use country only?
df['country']=df.location.str.split(',',expand=True)[:][0]

df['state']=df.location.str.split(',',expand=True)[:][1]
df['city']=df.location.str.split(',',expand=True)[:][2]

#----------2. replacing different kinds of missing value to np.nan
df.state = df.state.str.strip()
df.state.fillna(value=np.nan, inplace=True)
df.state.replace('', np.nan, inplace=True)
df.state.replace(' ', np.nan, inplace=True)


#df['state'].sort_values().unique()

#----------3. clear the whitespaces and signs at the start/end

df.city = df.city.str.strip(' /:\\')

#---------4. replacing different kinds of missing value to np.nan
df.city.fillna(value=np.nan, inplace=True)
df.city.replace('', np.nan, inplace=True)
df.city.replace(' ', np.nan, inplace=True)
df['city']=df.city.str.lower()
#------

# drop unnecessary columns
df.drop(['location'], axis = 1, inplace = True)

# too many empty fields in city, hence dropped
df.drop(columns=['city'], axis=1, inplace = True)

In [ ]:
#-------------------start cleaning for text columns

def preprocess_text(text):
    
    
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\b[A-Za-z]+\b')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords


df['text'] = df.apply(lambda row: (str(row['title']) + ' ' + 
                                   str(row['company_profile']) + ' ' + 
                                   str(row['description']) + ' ' + 
                                   str(row['requirements']) + ' ' + 
                                   str(row['benefits'])), axis = 1)

# Fill empty columns with "Unspecified"
df['text'] = df['text'].fillna("Unspecified")
    
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text, ngram_range = (1,2),   #TRY ADJUSTING ngram_range and min_df, max_df
                                min_df= 0.01, max_df= 0.7)

# Fit to the data and transform to feature matrix
text_column = vectoriser.fit_transform(df['text'])

# Convert sparse matrix to dataframe
text_column = pd.DataFrame.sparse.from_spmatrix(text_column)

# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}

# Rename each column using the mapping
for col in text_column.columns:
    text_column.rename(columns={col: col_map[col]}, inplace=True)

In [ ]:
#-------------------continue cleaning for text columns

# identify which text feature is not populated before dropping them
columns_to_remove = ['requirements', 'company_profile', 'description', 'benefits', 'title', 'text']

for i in columns_to_remove:
  df[i] = df[i].fillna("unspecified")

# create new binary columns to check if features are provided in the job ad
new_columns = ['has_requirements', 'has_company_profile', 'has_description', 'has_benefits']

for i in range(len(new_columns)):
  df[new_columns[i]] = df[columns_to_remove[i]].apply(lambda x: 0 if x == "unspecified" else 1)

df.drop(columns = columns_to_remove, axis = 1, inplace = True)
df.drop(columns = ['job_id'], axis = 1, inplace = True)

In [ ]:
#-------------------cleaning categorical columns 
category_columns = ['employment_type', 'required_experience', 'required_education', 'function', 'country', 'state']

for i in category_columns:
  df[i] = df[i].fillna(i + "_unspecified")

categorical_features = pd.concat([pd.get_dummies(df['employment_type']), 
                                  pd.get_dummies(df['required_experience']), 
                                  pd.get_dummies(df['required_education']), 
                                  pd.get_dummies(df['country']),
                                  pd.get_dummies(df['state'])])

df.drop(columns = category_columns, axis = 1, inplace = True)

In [ ]:
# combine vectorised columns and sparse categorical columns to the main df
df = pd.concat([df, text_column], axis =1, ignore_index=False)

In [ ]:
X_data = df.drop(columns = 'fraudulent', axis = 1)
y_data = df['fraudulent']

In [ ]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42, sampling_strategy=0.4)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
X_resample, y_resampled = smt.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


In [ ]:
clf = ensemble.RandomForestClassifier(random_state=42)
clf.fit(X_resample, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


RandomForestClassifier(random_state=42)

In [ ]:
y_pred_train = clf.predict(X_resample)
y_pred_prob_train = clf.predict_proba(X_resample)[:,1]
    
precision_train = metrics.precision_score(y_resampled, y_pred_train)
recall_train = metrics.recall_score(y_resampled, y_pred_train)
f1_train = metrics.f1_score(y_resampled, y_pred_train)
roc_auc_train = metrics.roc_auc_score(y_resampled, y_pred_prob_train)

precision_train, recall_train, f1_train, roc_auc_train

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


(1.0, 1.0, 1.0, 1.0)

In [ ]:
y_pred_test = clf.predict(X_test)
y_pred_prob_test = clf.predict_proba(X_test)[:,1]
    
precision = metrics.precision_score(y_test, y_pred_test)
recall = metrics.recall_score(y_test, y_pred_test)
f1 = metrics.f1_score(y_test, y_pred_test)
roc_auc = metrics.roc_auc_score(y_test, y_pred_prob_test)

precision, recall, f1, roc_auc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


(1.0, 0.6187845303867403, 0.7645051194539249, 0.9877769550606595)

In [ ]:
rf = {'Feature': clf.feature_names_in_, 'Importance': clf.feature_importances_}
df_rf = pd.DataFrame(rf)
df_rf = df_rf.sort_values(by = 'Importance', key = abs, ascending = False, ignore_index =True)
df_rf.head(10)

,Feature,Importance
0,has_company_logo,0.036117
1,has_company_profile,0.030755
2,nan,0.021251
3,earn,0.020401
4,administrative,0.009281
5,clerk,0.008873
6,follow,0.008391
7,per,0.008291
8,clerical,0.007495
9,gas,0.006956


In [ ]:
def model_build(base_estimator, parameters):
    
    clf = model_selection.RandomizedSearchCV(
        base_estimator, 
        parameters, 
        scoring = 'recall')
    
    clf.fit(X_resample, y_resampled)
    
    print('best score:', clf.best_score_)
    print('best parameters: ', clf.best_params_)

    optimised = clf.best_estimator_
    return optimised

In [ ]:
random_grid = {'n_estimators': [x for x in np.arange(start=80,stop=200,step=10)],
               'criterion': ['gini', 'entropy'], 
               'min_samples_leaf': [1,2,3,4,5],
               'min_samples_split': [2,3,4,5]}

optimised_rf = model_build(ensemble.RandomForestClassifier(random_state=2022), random_grid)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse co

best score: 0.9812803598958728
best parameters:  {'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}


In [ ]:
rf = {'Feature': optimised_rf.feature_names_in_, 'Importance': optimised_rf.feature_importances_}
df_rf = pd.DataFrame(rf)
df_rf = df_rf.sort_values(by = 'Importance', ascending = False, ignore_index =True)
df_rf.head(10)

,Feature,Importance
0,has_company_profile,0.041066
1,has_company_logo,0.038099
2,nan,0.019175
3,earn,0.014921
4,oil,0.011162
5,per,0.010413
6,entry,0.009335
7,schedule,0.007972
8,clerk,0.007865
9,administrative,0.007640


In [ ]:
y_pred_train = optimised_rf.predict(X_resample)
y_pred_prob_train = optimised_rf.predict_proba(X_resample)[:,1]
    
precision_train = metrics.precision_score(y_resampled, y_pred_train)
recall_train = metrics.recall_score(y_resampled, y_pred_train)
f1_train = metrics.f1_score(y_resampled, y_pred_train)
roc_auc_train = metrics.roc_auc_score(y_resampled, y_pred_prob_train)

precision_train, recall_train, f1_train, roc_auc_train

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


(1.0, 0.9996328254084818, 0.999816378993757, 1.0)

In [ ]:
y_pred_test = optimised_rf.predict(X_test)
y_pred_prob_test = optimised_rf.predict_proba(X_test)[:,1]
    
precision = metrics.precision_score(y_test, y_pred_test)
recall = metrics.recall_score(y_test, y_pred_test)
f1 = metrics.f1_score(y_test, y_pred_test)
roc_auc = metrics.roc_auc_score(y_test, y_pred_prob_test)

precision, recall, f1, roc_auc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


(0.9823008849557522,
 0.6132596685082873,
 0.7551020408163266,
 0.9872521338660201)

In [ ]:
optimised_rf

RandomForestClassifier(min_samples_split=5, n_estimators=80, random_state=2022)

In [ ]:
# build Naive Bayes Model
mnb = naive_bayes.MultinomialNB()
mnb.fit(X_resample, y_resampled)

In [ ]:
y_pred_train = mnb.predict(X_resample)
y_pred_prob_train = mnb.predict_proba(X_resample)[:,1]
    
precision_train = metrics.precision_score(y_resampled, y_pred_train)
recall_train = metrics.recall_score(y_resampled, y_pred_train)
f1_train = metrics.f1_score(y_resampled, y_pred_train)
roc_auc_train = metrics.roc_auc_score(y_resampled, y_pred_prob_train)

precision_train, recall_train, f1_train, roc_auc_train

In [ ]:
y_pred_test = mnb.predict(X_test)
y_pred_prob_test = mnb.predict_proba(X_test)[:,1]
    
precision = metrics.precision_score(y_test, y_pred_test)
recall = metrics.recall_score(y_test, y_pred_test)
f1 = metrics.f1_score(y_test, y_pred_test)
roc_auc = metrics.roc_auc_score(y_test, y_pred_prob_test)

precision, recall, f1, roc_auc

In [ ]:
from sklearn import metrics

cf_matrixtr = metrics.confusion_matrix(y_resampled, y_pred_train)
cf_matrixte = metrics.confusion_matrix(y_test, y_pred_test)

report = """
Confusion Matrix for train dataset:
{}
Confusion Matrix for test dataset:
{}
""".format(cf_matrixtr, cf_matrixte)
print(report)

In [ ]:
# match feature names and respective feature counts in each class
f = {'Feature': X_resample.columns, 'fraud': mnb.feature_count_[1,:],
         'not_fraud': mnb.feature_count_[0,:]}
features = pd.DataFrame(data=f)

# calculate the P(feature|class) by using feature count in the class divided by respective class count
# 1 count is added to all feature counts to ensure ratio can be calculated later
features['P (f|fraud)'] = (features['fraud'] +1) / mnb.class_count_[1]
features['P (f|not_fraud)'] = (features['not_fraud'] +1) / mnb.class_count_[0]

# ratio of the P(feature|class) is used to evaluate the predictiveness of the token 
features['Ratio'] = features['P (f|fraud)'] / features['P (f|not_fraud)']

# print out the top 10 features for each class
print('The top 10 most predictive tokens for fraud job postings are:')
print(features.sort_values(by=['Ratio'], ascending= False).head(20)['Feature'].values)

print('The top 10 most predictive tokens for not fraud job postings are:')
print(features.sort_values(by=['Ratio'], ascending= False).tail(20)['Feature'].values)

In [ ]:
# try naive bayes with only text columns
X_train, X_test, y_train, y_test = train_test_split(df_trial, y_trial, test_size=0.2, random_state=42)
X_resample, y_resampled = smt.fit_resample(X_train, y_train)
text_resample = X_resample.drop(X_resample.columns[0:8], axis = 1)
text_test = X_test.drop(X_resample.columns[0:8], axis = 1)

In [ ]:
mnb2 = naive_bayes.MultinomialNB()
mnb2.fit(text_resample, y_resampled)

y_pred_train = mnb.predict(X_resample)
y_pred_prob_train = mnb.predict_proba(X_resample)[:,1]
y_pred_test = mnb2.predict(text_test)
y_pred_prob_test = mnb2.predict_proba(text_test)[:,1]
    
precision = metrics.precision_score(y_test, y_pred_test)
recall = metrics.recall_score(y_test, y_pred_test)
f1 = metrics.f1_score(y_test, y_pred_test)
roc_auc = metrics.roc_auc_score(y_test, y_pred_prob_test)

precision, recall, f1, roc_auc

In [ ]:
from sklearn import metrics

cf_matrixtr = metrics.confusion_matrix(y_resampled, y_pred_train)
cf_matrixte = metrics.confusion_matrix(y_test, y_pred_test)

report = """
Confusion Matrix for train dataset:
{}
Confusion Matrix for test dataset:
{}
""".format(cf_matrixtr, cf_matrixte)
print(report)

In [ ]:

y_score = mnb.predict_proba(X_test)[:, 1]

#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, y_score)

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()